In [ ]:
%%capture
!pip install datasets
!pip install optuna

In [ ]:
# The value for the seed and paths
SEED = 168
PATH_TO_W2V_MODEL_DRIVE = '/content/drive/MyDrive/NLP 24-25 Sem1/GoogleNews-vectors-negative300.bin'
PATH_TO_W2V_MODEL_LOCAL = r'C:/Users/60178/Downloads/GoogleNews-vectors-negative300.bin'
PATH_TO_FASTTEXT_MODEL_DRIVE = '/content/drive/MyDrive/NLP 24-25 Sem1/cc.en.300.bin'
PATH_TO_FASTTEXT_MODEL_LOCAL = r'C:/Users/60178/Downloads/cc.en.300.bin/cc.en.300.bin'

In [ ]:
# Standard library imports
from collections import Counter

# General third-party libraries import
import copy
from gensim.models import KeyedVectors
from gensim.models.fasttext import load_facebook_model
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import random
import string
import sys

# Import optuna for hyperparams tuning
import optuna

# Import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary nltk resources
nltk.download('wordnet')
nltk.download('punkt')  # Ensure punkt is downloaded for tokenization
nltk.download('stopwords')  # Ensure stopwords are also downloaded if not already

# Import pytorch
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn.utils.rnn import pad_sequence

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Method to set random seed to ensure consistency
def set_seed(seed = SEED):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# Factory to create the dataloader
def dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device):
    # Create Tensor datasets
    train_dataset = TensorDataset(X_train_sequence.to(device), torch.tensor(Y_train, dtype=torch.long, device=device))
    val_dataset = TensorDataset(X_val_sequence.to(device), torch.tensor(Y_val, dtype=torch.long, device=device))
    test_dataset = TensorDataset(X_test_sequence.to(device), torch.tensor(Y_test, dtype=torch.long, device=device))

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader

# Early Stopper
class EarlyStopper:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.max_validation_accuracy = 0

    def early_stop(self, validation_accuracy):
        if validation_accuracy > self.max_validation_accuracy:
            self.max_validation_accuracy = validation_accuracy
            self.counter = 0
        elif validation_accuracy < (self.max_validation_accuracy + self.delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

# Factory to provide required optimizer which specified parameters
def optimizer_factory(optimizer_name, model, learning_rate, momentum = 0.8):
    if optimizer_name == 'Adam':
        return torch.optim.Adam(model.parameters(), lr = learning_rate)
    elif optimizer_name == 'SGD':
        return torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = momentum)
    elif optimizer_name == 'RMSprop':
        return torch.optim.RMSprop(model.parameters(), lr = learning_rate)
    elif optimizer_name == 'AdamW':
        return torch.optim.AdamW(model.parameters(), lr=learning_rate)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

# Method to train the model
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()  # Set model to training mode
    train_loss = 0  # To accumulate loss for this epoch

    for inputs, labels in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        # Calculate loss
        loss = loss_fn(outputs.squeeze(), labels.float())  # Squeeze to match dimensions
        loss.backward()  # Backpropagation

        # # Apply gradient clipping
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        optimizer.step()  # Update parameters

        train_loss += loss.item()  # Accumulate loss

    # Calculate average loss for the epoch
    train_loss /= len(dataloader)
    return train_loss

# Method to evaluate the model
def test_loop(dataloader, model):
    model.eval() # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in dataloader:
            outputs = model(inputs)  # Forward pass
            predicted = (outputs.squeeze() > 0.5).float()  # Apply threshold

            total += labels.size(0)
            correct += (predicted == labels.float()).sum().item()  # Count correct predictions

    accuracy = correct / total
    return accuracy

# Method to plot the performance graph
def plot_performance(train_losses, val_accuracies, num_epochs):
    # Visualization
    plt.figure(figsize=(12, 5))

    # Plotting Loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, num_epochs + 1, 10))
    plt.legend()

    # Plotting Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='orange')
    plt.title('Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.xticks(range(1, num_epochs + 1, 10))
    plt.legend()

    plt.tight_layout()
    plt.show()

# **Part 0. Dataset Preparation**

Part 0. Dataset Preparation We will be using the movie review dataset introduced in https://www.cs.cornell.edu/people/ pabo/movie-review-data/rt-polaritydata.README.1.0.txt. To load this dataset, you need to install the “datasets” library via pip install datasets. Then you can use the following code snippet: 1 from datasets import load_dataset 2 dataset = load_dataset (" rotten_tomatoes ") 3 train_dataset = dataset [’train ’] 4 validation_dataset = dataset [’validation ’] 5 test_dataset = dataset [’test ’] Using the original train-valid-test split provided in the above code, you will perform model training on the training dataset, configure your model (e.g., learning rate, batch size, number of training epochs) on the validation dataset, and conduct evaluation on the test dataset.

# **Load Dataset**

In [ ]:
set_seed()
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
print(f"train_dataset size: {len(train_dataset)}")
print(f"validation_dataset size: {len(validation_dataset)}")
print(f"test_dataset size: {len(test_dataset)}")
print(f"data format: {train_dataset[0]}")

train_dataset size: 8530
validation_dataset size: 1066
test_dataset size: 1066
data format: {'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}


In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"The name of the CUDA device is: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA is available!
The name of the CUDA device is: Tesla T4


# **Load word2vec Model**

In [ ]:
# Load the pre-trained Word2Vec model
if 'google.colab' in sys.modules:
    print("Running on Google Colab's hosted runtime")
    from google.colab import drive
    drive.mount('/content/drive')
    w2v_model = KeyedVectors.load_word2vec_format(PATH_TO_W2V_MODEL_DRIVE, binary=True)
else:
    print("Running on a local runtime")
    w2v_model = KeyedVectors.load_word2vec_format(PATH_TO_W2V_MODEL_LOCAL, binary=True)

# Get the dimension of the embeddings
vector_dim = w2v_model.vector_size
print(f"Word2Vec Dimension: {vector_dim}")


Running on Google Colab's hosted runtime
Mounted at /content/drive
Word2Vec Dimension: 300


# **Data Preprocessing**

In [ ]:
# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))

# Dataset Preprocessing to tokenize and/or lemmatize
def preprocess_dataset(data, lemmatization = True):
    processed_sentences = []

    for entry in data:
        text = entry['text']
        # Tokenize the sentence
        tokens = word_tokenize(text)
        if lemmatization:
          # With case folding, punctuation and stop words removal
          # tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word not in punctuation and word.lower() not in stop_words]

          # With case folding and punctuation removal
            tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word not in punctuation]

        processed_sentences.append(tokens)

    return processed_sentences

# Get vocab and oov words
def get_vocab_OOV(sentences, w2v_model):
    vocabulary = set()
    oov_words = set()

    for sentence in sentences:
        for word in sentence:
            vocabulary.add(word)
            if word not in w2v_model:
                oov_words.add(word)

    return vocabulary, oov_words

vocabulary, oov_words = get_vocab_OOV(preprocess_dataset(train_dataset, False), w2v_model)
lemmatized_vocabulary, lemmatized_oov_words = get_vocab_OOV(preprocess_dataset(train_dataset), w2v_model)

print(f"Before lemmatization: Vocabulary size: {len(vocabulary)}, OOV words: {len(oov_words)}")
print(f"After lemmatization: Vocabulary size: {len(lemmatized_vocabulary)}, OOV words: {len(lemmatized_oov_words)}")

Before lemmatization: Vocabulary size: 18030, OOV words: 3613
After lemmatization: Vocabulary size: 16565, OOV words: 3584




1.   Lemmatize, case folding and remove stop words for train, validation and test set
2.   Drop words not found in train dataset for validation and test set




In [ ]:
preprocessed_train_dataset = preprocess_dataset(train_dataset)
preprocessed_validation_dataset = preprocess_dataset(validation_dataset)
preprocessed_test_dataset = preprocess_dataset(test_dataset)

In [ ]:
def drop_oov(preprocessed_dataset, w2v_model):
    return [[word for word in sentence if word in w2v_model] for sentence in preprocessed_dataset]

def drop_non_train_vocab(dataset, train_vocab):
    return [[word for word in sentence if word in train_vocab] for sentence in dataset]

X_train = preprocessed_train_dataset
# X_train = drop_oov(preprocessed_train_dataset, w2v_model)
Y_train = [entry['label'] for entry in train_dataset]

# X_val = drop_non_train_vocab(drop_oov(preprocessed_validation_dataset, w2v_model), lemmatized_vocabulary)
X_val = drop_non_train_vocab(preprocessed_validation_dataset, lemmatized_vocabulary)
# X_val = preprocessed_validation_dataset
Y_val = [entry['label'] for entry in validation_dataset]

# X_test = drop_non_train_vocab(drop_oov(preprocessed_test_dataset, w2v_model), lemmatized_vocabulary)
X_test = drop_non_train_vocab(preprocessed_test_dataset, lemmatized_vocabulary)
# X_test = preprocessed_test_dataset
Y_test = [entry['label'] for entry in test_dataset]


Sanity check: If preprocessed data is wrong

In [ ]:
print(f"X_train size: {len(X_train)}")
print(f"Y_train size: {len(Y_train)}")

print(f"X_val size: {len(X_val)}")
print(f"Y_val size: {len(Y_val)}")

print(f"X_test size: {len(X_test)}")
print(f"Y_test size: {len(Y_test)}")

X_train size: 8530
Y_train size: 8530
X_val size: 1066
Y_val size: 1066
X_test size: 1066
Y_test size: 1066


# **Generate Embedding**

In [ ]:
def get_word2index(train_vocab):
    word2index = {}
    for idx, word in enumerate(train_vocab):
        word2index[word] = idx + 1  # +1 to offset 0 for padding
    return word2index

word2index = get_word2index(lemmatized_vocabulary)

In [ ]:
print(f"word2index size = vocab size: {len(word2index)}")

word2index size = vocab size: 16565


In [ ]:
def create_embedding_matrix(w2v_model, word2index, embedding_dim=300):
    """
    Creates an embedding matrix based on the word2index mapping and Word2Vec model.

    Args:
    - w2v_model: Pre-trained Word2Vec model.
    - word2index: Dictionary mapping words to their indices.
    - embedding_dim: Dimension of the Word2Vec word vectors (default: 300).

    Returns:
    - embedding_matrix: Embedding matrix where each row corresponds to the vector of a word in the vocabulary.
    """
    vocab_size = len(word2index)  # Number of words in the training vocabulary

    # Initialize the embedding matrix with zeros (or any other value)
    embedding_matrix = np.zeros((vocab_size + 1, embedding_dim))
    mean = np.mean(w2v_model.vectors, axis=0)
    std = np.std(w2v_model.vectors, axis=0)

    for word, idx in word2index.items():
        if word in w2v_model:
            # If the word exists in the Word2Vec model, use its pre-trained embedding
            embedding_matrix[idx] = w2v_model[word]
        else:
            embedding_matrix[idx] = np.random.normal(
                loc=mean, scale=std, size=(embedding_dim,)
            )

    return embedding_matrix

embedding_dim = w2v_model.vector_size  # Get the dimension of the word vectors in Word2Vec
embedding_matrix = create_embedding_matrix(w2v_model, word2index, embedding_dim)
print(f"embedding_matrix shape: {embedding_matrix.shape}")


embedding_matrix shape: (16566, 300)


# **Sentence2sequence + Padding**

In [ ]:
# Get maximum sentence length
def get_max_sentence_length(*datasets):
    """
    Get the maximum sentence length from the provided datasets.

    Args:
    - datasets: A variable number of lists containing tokenized sentences.

    Returns:
    - max_length: The maximum sentence length across all datasets.
    """
    max_length = 0
    for index, dataset in enumerate(datasets):
        # Update max_length if a longer sentence is found
        max_length = max(max_length, max(len(sentence) for sentence in dataset))
        min_length = min(len(sentence) for sentence in dataset)
        print(f"Min sentence length in dataset {index}: {min_length}")
    return max_length

# Example usage
max_length = get_max_sentence_length(X_train, X_val, X_test)
print(f"Maximum sentence length of three datasets: {max_length}")

Min sentence length in dataset 0: 1
Min sentence length in dataset 1: 1
Min sentence length in dataset 2: 2
Maximum sentence length of three datasets: 53


In [ ]:
def convert_to_sequence_data(tokenized_data, word2index, max_length=max_length):
    """
    Preprocess the tokenized data for RNN input.

    Args:
    - tokenized_data: List of tokenized sentences.
    - word2index: Dictionary mapping words to indices.
    - max_length: Maximum length for padding (default is 100).

    Returns:
    - padded_sequences: Padded tensor of sequences.
    """
    # Convert tokens to indices
    indexed_sequences = []
    for sentence in tokenized_data:
        indexed_sentence = [word2index.get(word, 0) for word in sentence]  # Use 0 for OOV words

        # Truncate or pad the indexed sentence to the specified max_length
        if len(indexed_sentence) > max_length:
            indexed_sentence = indexed_sentence[:max_length]  # Truncate
        else:
            indexed_sentence += [0] * (max_length - len(indexed_sentence))  # Pad with zeros

        indexed_sequences.append(torch.tensor(indexed_sentence, dtype=torch.long))

    # Convert list of tensors to a padded tensor
    padded_sequences = torch.stack(indexed_sequences)

    return padded_sequences

# Convert to sequence
X_train_sequence = convert_to_sequence_data(X_train, word2index)
X_val_sequence = convert_to_sequence_data(X_val , word2index)
X_test_sequence = convert_to_sequence_data(X_test, word2index)


Sanity check: if the sequenced tokens mapped correctly to embedding matrix and w2v_model

In [ ]:
# Check if the sequenced tokens mapped correctly to embedding matrix and w2v_model
embedding_matrix[X_train_sequence[0][0]] == w2v_model[X_train[0][0]]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
print(f"Shape of X_train_sequence: {X_train_sequence.shape}")
print(f"Shape of X_valn_sequence: {X_val_sequence.shape}")
print(f"Shape of X_test_sequence: {X_test_sequence.shape}")

Shape of X_train_sequence: torch.Size([8530, 53])
Shape of X_valn_sequence: torch.Size([1066, 53])
Shape of X_test_sequence: torch.Size([1066, 53])


# biGRU (keep padding)

In [ ]:
class BiGRU(nn.Module):
    def __init__(self, embedding_matrix, hidden_size=64, num_gru_layers=1, bidirectional=True, aggregation='max_pool'):
        super(BiGRU, self).__init__()
        embedding_dim = embedding_matrix.shape[1]
        self.aggregation = aggregation

        # Embedding layer using the provided embedding matrix
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            padding_idx=0,  # Assuming 0 is the padding index
            freeze=False  # Freezing the embeddings
        )

        # Dropout layer
        self.dropout = nn.Dropout(0.2)

        # GRU layer(s)
        self.gru = nn.GRU(input_size=embedding_dim,
                          hidden_size=hidden_size,
                          num_layers=num_gru_layers,
                          dropout=(0 if num_gru_layers == 1 else 0.2),
                          bidirectional=bidirectional,
                          batch_first=True)

        # Linear layer for binary classification
        # Output size is 1 for binary classification
        num_directions = 2 if bidirectional else 1
        is_max_mean = 2 if aggregation == 'max_mean' else 1
        self.fc = nn.Linear(hidden_size * num_directions * is_max_mean, 1)

        if aggregation == 'mhsa':
            self.mhsa = nn.MultiheadAttention(embed_dim=hidden_size * num_directions, num_heads=4, batch_first=True)

    def forward(self, x, hidden = None):
        # Pass through the embedding layer
        embedded = self.embedding(x)

        # Pass through the GRU layer(s)
        gru_out, _ = self.gru(embedded, hidden)

        # Pooling over the time dimension (sequence length)
        if self.aggregation == 'max_pool':
            gru_out, _ = torch.max(gru_out, dim=1)  # Max-pooling over the sequence length
        elif self.aggregation == 'mean_pool':
            gru_out = torch.mean(gru_out, dim=1)  # Mean-pooling over the sequence length
        elif self.aggregation == 'max_mean':
            max_pooled, _ = torch.max(gru_out, dim=1)  # Max-pooling over the sequence length
            mean_pooled = torch.mean(gru_out, dim=1)   # Mean-pooling over the sequence length
            # Concatenate max and mean pooled vectors
            gru_out = torch.cat((max_pooled, mean_pooled), dim=1)
        elif self.aggregation == 'mhsa':
            gru_out, _ = self.mhsa(gru_out, gru_out, gru_out)
            gru_out = torch.mean(gru_out, dim=1)  # Mean-pooling over the sequence length

        # Apply dropout
        gru_out = self.dropout(gru_out)

        # Pass through the fully connected layer and apply sigmoid activation
        out = self.fc(gru_out)
        out = torch.sigmoid(out)

        return out

# Hyperparameter Tuning

In [ ]:
def bigru_w2v_objective(trial):

    max_accuracy = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Hyperparameters
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])  # Hidden size choices
    num_gru_layers = trial.suggest_categorical('num_gru_layers', [1, 2, 3])  # Number of GRU layers
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])  # Learning rate choices
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size choices
    num_epochs = trial.suggest_categorical('num_epochs', [10, 20, 30, 40, 50])  # Number of epochs
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop', 'AdamW'])  # Optimizer choices

    # Load data using the dataloader_factory
    train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

    # Define the BiGRU model
    model = BiGRU(embedding_matrix, hidden_size, num_gru_layers, True, 'mean_pool')
    model.to(device)
    criterion = nn.BCELoss()  # Use BCELoss for binary classification

    # Choose optimizer based on trial suggestion
    optimizer = optimizer_factory(optimizer_name, model, learning_rate)

    # Lists to store loss and accuracy for visualization
    train_losses = []
    val_accuracies = []

    # Training Loop
    for epoch in range(num_epochs):
        loss = train_loop(train_loader, model, criterion, optimizer)
        train_losses.append(loss)

        # Evaluate on validation set
        val_accuracy = test_loop(val_loader, model)
        val_accuracies.append(val_accuracy)
        if val_accuracy > max_accuracy:
            max_accuracy = val_accuracy

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    return max_accuracy  # Return validation accuracy for optimization

In [ ]:
set_seed()
study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(bigru_w2v_objective, n_trials=50)  # Invoke optimization of the objective function.

[I 2024-11-01 16:49:22,314] A new study created in memory with name: no-name-5dfd0def-e2c7-4dda-ad42-4cd4e627c921


Epoch [1/10], Train Loss: 0.6950, Validation Accuracy: 0.5000
Epoch [2/10], Train Loss: 0.6933, Validation Accuracy: 0.5000
Epoch [3/10], Train Loss: 0.6924, Validation Accuracy: 0.5028
Epoch [4/10], Train Loss: 0.6910, Validation Accuracy: 0.5178
Epoch [5/10], Train Loss: 0.6901, Validation Accuracy: 0.5460
Epoch [6/10], Train Loss: 0.6890, Validation Accuracy: 0.5779
Epoch [7/10], Train Loss: 0.6878, Validation Accuracy: 0.6079
Epoch [8/10], Train Loss: 0.6862, Validation Accuracy: 0.6285
Epoch [9/10], Train Loss: 0.6849, Validation Accuracy: 0.6576


[I 2024-11-01 16:49:41,222] Trial 0 finished with value: 0.6651031894934334 and parameters: {'hidden_size': 64, 'num_gru_layers': 1, 'learning_rate': 1e-05, 'batch_size': 32, 'num_epochs': 10, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.6651031894934334.


Epoch [10/10], Train Loss: 0.6831, Validation Accuracy: 0.6651
Epoch [1/50], Train Loss: 36.4311, Validation Accuracy: 0.5103
Epoch [2/50], Train Loss: 1.5626, Validation Accuracy: 0.5047
Epoch [3/50], Train Loss: 1.2303, Validation Accuracy: 0.5235
Epoch [4/50], Train Loss: 0.9672, Validation Accuracy: 0.5263
Epoch [5/50], Train Loss: 0.8131, Validation Accuracy: 0.5009
Epoch [6/50], Train Loss: 1.3473, Validation Accuracy: 0.5422
Epoch [7/50], Train Loss: 1.0469, Validation Accuracy: 0.5019
Epoch [8/50], Train Loss: 0.7759, Validation Accuracy: 0.5328
Epoch [9/50], Train Loss: 0.9199, Validation Accuracy: 0.5516
Epoch [10/50], Train Loss: 0.7834, Validation Accuracy: 0.5000
Epoch [11/50], Train Loss: 0.8189, Validation Accuracy: 0.5000
Epoch [12/50], Train Loss: 0.9917, Validation Accuracy: 0.5188
Epoch [13/50], Train Loss: 0.9666, Validation Accuracy: 0.5038
Epoch [14/50], Train Loss: 0.8241, Validation Accuracy: 0.5169
Epoch [15/50], Train Loss: 1.0337, Validation Accuracy: 0.5000


[I 2024-11-01 16:51:26,671] Trial 1 finished with value: 0.5769230769230769 and parameters: {'hidden_size': 256, 'num_gru_layers': 2, 'learning_rate': 0.1, 'batch_size': 128, 'num_epochs': 50, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.6651031894934334.


...

Epoch [20/20], Train Loss: 0.0000, Validation Accuracy: 0.7486
Epoch [1/40], Train Loss: 0.6135, Validation Accuracy: 0.7364
Epoch [2/40], Train Loss: 0.4717, Validation Accuracy: 0.7523
Epoch [3/40], Train Loss: 0.4118, Validation Accuracy: 0.7730
Epoch [4/40], Train Loss: 0.3692, Validation Accuracy: 0.7739
Epoch [5/40], Train Loss: 0.3330, Validation Accuracy: 0.7561
Epoch [6/40], Train Loss: 0.3025, Validation Accuracy: 0.7814
Epoch [7/40], Train Loss: 0.2774, Validation Accuracy: 0.7786
Epoch [8/40], Train Loss: 0.2476, Validation Accuracy: 0.7767
Epoch [9/40], Train Loss: 0.2263, Validation Accuracy: 0.7805
Epoch [10/40], Train Loss: 0.2006, Validation Accuracy: 0.7842
Epoch [11/40], Train Loss: 0.1815, Validation Accuracy: 0.7824
Epoch [12/40], Train Loss: 0.1581, Validation Accuracy: 0.7805
Epoch [13/40], Train Loss: 0.1396, Validation Accuracy: 0.7805
Epoch [14/40], Train Loss: 0.1337, Validation Accuracy: 0.7655
Epoch [15/40], Train Loss: 0.1159, Validation Accuracy: 0.7655
E

[I 2024-11-01 17:23:04,166] Trial 48 finished with value: 0.7842401500938087 and parameters: {'hidden_size': 256, 'num_gru_layers': 1, 'learning_rate': 0.0001, 'batch_size': 128, 'num_epochs': 40, 'optimizer': 'RMSprop'}. Best is trial 27 with value: 0.801125703564728.


Epoch [40/40], Train Loss: 0.0053, Validation Accuracy: 0.7561
Epoch [1/10], Train Loss: 0.5569, Validation Accuracy: 0.7749
Epoch [2/10], Train Loss: 0.3107, Validation Accuracy: 0.7711
Epoch [3/10], Train Loss: 0.1640, Validation Accuracy: 0.7786
Epoch [4/10], Train Loss: 0.0842, Validation Accuracy: 0.7561
Epoch [5/10], Train Loss: 0.0447, Validation Accuracy: 0.7523
Epoch [6/10], Train Loss: 0.0276, Validation Accuracy: 0.7617
Epoch [7/10], Train Loss: 0.0164, Validation Accuracy: 0.7533
Epoch [8/10], Train Loss: 0.0111, Validation Accuracy: 0.7589
Epoch [9/10], Train Loss: 0.0070, Validation Accuracy: 0.7439


[I 2024-11-01 17:23:12,028] Trial 49 finished with value: 0.7786116322701688 and parameters: {'hidden_size': 128, 'num_gru_layers': 1, 'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 10, 'optimizer': 'Adam'}. Best is trial 27 with value: 0.801125703564728.


Epoch [10/10], Train Loss: 0.0088, Validation Accuracy: 0.7495


# Best Hyperparameter

In [ ]:
# The Best Hyperparameters
hidden_size = 128  # Hidden size choices
num_gru_layers = 1  # Number of GRU layers choices
learning_rate = 0.001  # Learning rate choices
batch_size = 64  # Batch size choices
num_epochs = 40  # num epoch choices
optimizer_name = 'RMSprop'  # Optimizer choices

In [ ]:
# Reset the seed to ensure fairness when comparing performances
set_seed()

# Load Data
train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

# Initialize the model
model = BiGRU(embedding_matrix, hidden_size, num_gru_layers, True, 'mean_pool')
model.to(device)
# Use BCELoss for binary classification
criterion = nn.BCELoss()
# Choose optimizer based on trial suggestion
optimizer = optimizer_factory(optimizer_name, model, learning_rate)
# Initialize the early stopper
early_stopper = EarlyStopper()

# Lists to store loss and accuracy for visualization
train_losses = []
val_accuracies = []

# Training Loop
best_val_accuracy = 0
best_model = None
for epoch in range(num_epochs):
    # Train on train set
    train_loss = train_loop(train_loader, model, criterion, optimizer)
    train_losses.append(train_loss)

    # Evaluate on validation set
    val_accuracy = test_loop(val_loader, model)
    val_accuracies.append(val_accuracy)

    # Get the best model
    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_model = copy.deepcopy(model)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping
    if early_stopper.early_stop(val_accuracy):
        print("Early stopping triggered!")
        break

# Evaluate on test data
test_accuracy = test_loop(test_loader, best_model)

# Print out the performance
print(f"Using Best Model, Test Accuracy: {test_accuracy:.4f}")

Epoch [1/40], Train Loss: 0.5439, Validation Accuracy: 0.7711
Epoch [2/40], Train Loss: 0.3058, Validation Accuracy: 0.7814
Epoch [3/40], Train Loss: 0.1811, Validation Accuracy: 0.7842
Epoch [4/40], Train Loss: 0.0977, Validation Accuracy: 0.7758
Epoch [5/40], Train Loss: 0.0495, Validation Accuracy: 0.7655
Epoch [6/40], Train Loss: 0.0280, Validation Accuracy: 0.7505
Epoch [7/40], Train Loss: 0.0147, Validation Accuracy: 0.7561
Epoch [8/40], Train Loss: 0.0073, Validation Accuracy: 0.7373
Epoch [9/40], Train Loss: 0.0054, Validation Accuracy: 0.7552
Epoch [10/40], Train Loss: 0.0036, Validation Accuracy: 0.7598
Epoch [11/40], Train Loss: 0.0034, Validation Accuracy: 0.7533
Epoch [12/40], Train Loss: 0.0014, Validation Accuracy: 0.7054
Epoch [13/40], Train Loss: 0.0047, Validation Accuracy: 0.7392
Early stopping triggered!
Using Best Model, Test Accuracy: 0.8021


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:1392: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1410.)
  result = _VF.gru(


# biGRU (mask padding)

In [ ]:
class BiGRU(nn.Module):
    def __init__(self, embedding_matrix, hidden_size=64, num_gru_layers=1, bidirectional=True, aggregation='max_pool'):
        super(BiGRU, self).__init__()
        embedding_dim = embedding_matrix.shape[1]
        self.aggregation = aggregation

        # Embedding layer using the provided embedding matrix
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            padding_idx=0,  # Assuming 0 is the padding index
            freeze=False  # Freezing the embeddings
        )

        # Dropout layer
        self.dropout = nn.Dropout(0.2)

        # GRU layer(s)
        self.gru = nn.GRU(input_size=embedding_dim,
                          hidden_size=hidden_size,
                          num_layers=num_gru_layers,
                          dropout=(0 if num_gru_layers == 1 else 0.2),
                          bidirectional=bidirectional,
                          batch_first=True)

        # Linear layer for binary classification
        # Output size is 1 for binary classification
        num_directions = 2 if bidirectional else 1
        is_max_mean = 2 if aggregation == 'max_mean' else 1
        self.fc = nn.Linear(hidden_size * num_directions * is_max_mean, 1)

        if aggregation == 'mhsa':
            self.mhsa = nn.MultiheadAttention(embed_dim=hidden_size * num_directions, num_heads=4, batch_first=True)

    def forward(self, x, hidden = None):
        # Pass through the embedding layer
        embedded = self.embedding(x)

        # Pass through the GRU layer(s)
        gru_out, _ = self.gru(embedded, hidden)

        # Pooling over the time dimension (sequence length)
        if self.aggregation == 'max_pool':
            mask = (x != self.embedding.padding_idx).unsqueeze(-1)
            gru_out = gru_out.masked_fill(~mask, -100000)
            gru_out, _ = torch.max(gru_out, dim=1)

        elif self.aggregation == 'mean_pool':
            mask = (x != self.embedding.padding_idx).unsqueeze(-1)
            gru_out = gru_out * mask
            summed = gru_out.sum(dim=1)
            valid_counts = mask.sum(dim=1)
            valid_counts = valid_counts.clamp(min=1)
            gru_out = summed / valid_counts

        elif self.aggregation == 'max_mean':
            mask = (x != self.embedding.padding_idx).unsqueeze(-1)
            max_pooled = gru_out.masked_fill(~mask, -100000)
            max_pooled, _ = torch.max(max_pooled, dim=1)

            mean_pooled = gru_out * mask
            summed = mean_pooled.sum(dim=1)
            valid_counts = mask.sum(dim=1)
            valid_counts = valid_counts.clamp(min=1)
            mean_pooled = summed / valid_counts

            # Concatenate max and mean pooled vectors
            gru_out = torch.cat((max_pooled, mean_pooled), dim=1)

        elif self.aggregation == 'mhsa':
            # Create a padding mask where 0 indicates a non-padding token and 1 indicates padding
            # This is a workaround to avoid receiving nan for all padded sentences
            padding_mask_final = torch.zeros(x.size(0), x.size(1), dtype=torch.bool, device=device)
            padding_mask = (x[:, 1:] == self.embedding.padding_idx)
            padding_mask_final[:, 1:] = padding_mask

            # Multi-head self-attention layer
            gru_out, _ = self.mhsa(gru_out, gru_out, gru_out, key_padding_mask=padding_mask_final)

            # Obtain the mean of the weighted output
            mask = (x != self.embedding.padding_idx).unsqueeze(-1)
            gru_out = gru_out * mask
            summed = gru_out.sum(dim=1)
            valid_counts = mask.sum(dim=1)
            valid_counts = valid_counts.clamp(min=1)
            gru_out = summed / valid_counts

        # Apply dropout
        gru_out = self.dropout(gru_out)

        # Pass through the fully connected layer and apply sigmoid activation
        out = self.fc(gru_out)
        out = torch.sigmoid(out)

        return out

# Hyperparameter Tuning

In [ ]:
def bigru_w2v_objective(trial):

    max_accuracy = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Hyperparameters
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])  # Hidden size choices
    num_gru_layers = trial.suggest_categorical('num_gru_layers', [1, 2, 3])  # Number of GRU layers
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])  # Learning rate choices
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size choices
    num_epochs = trial.suggest_categorical('num_epochs', [10, 20, 30, 40, 50])  # Number of epochs
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop', 'AdamW'])  # Optimizer choices

    # Load data using the dataloader_factory
    train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

    # Define the BiGRU model
    model = BiGRU(embedding_matrix, hidden_size, num_gru_layers, True, 'max_mean')
    model.to(device)
    criterion = nn.BCELoss()  # Use BCELoss for binary classification

    # Choose optimizer based on trial suggestion
    optimizer = optimizer_factory(optimizer_name, model, learning_rate)

    # Lists to store loss and accuracy for visualization
    train_losses = []
    val_accuracies = []

    # Training Loop
    for epoch in range(num_epochs):
        loss = train_loop(train_loader, model, criterion, optimizer)
        train_losses.append(loss)

        # Evaluate on validation set
        val_accuracy = test_loop(val_loader, model)
        val_accuracies.append(val_accuracy)
        if val_accuracy > max_accuracy:
            max_accuracy = val_accuracy

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    return max_accuracy  # Return validation accuracy for optimization

In [ ]:
set_seed()
study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(bigru_w2v_objective, n_trials=50)  # Invoke optimization of the objective function.

[I 2024-11-02 06:42:08,859] A new study created in memory with name: no-name-0fabea84-9f0b-47be-9970-56e7cc299a16


Epoch [1/20], Train Loss: 0.4966, Validation Accuracy: 0.8077
Epoch [2/20], Train Loss: 0.2567, Validation Accuracy: 0.7927
Epoch [3/20], Train Loss: 0.1134, Validation Accuracy: 0.7608
Epoch [4/20], Train Loss: 0.0450, Validation Accuracy: 0.7861
Epoch [5/20], Train Loss: 0.0190, Validation Accuracy: 0.7711
Epoch [6/20], Train Loss: 0.0117, Validation Accuracy: 0.7674
Epoch [7/20], Train Loss: 0.0056, Validation Accuracy: 0.7786
Epoch [8/20], Train Loss: 0.0053, Validation Accuracy: 0.7702
Epoch [9/20], Train Loss: 0.0025, Validation Accuracy: 0.7739
Epoch [10/20], Train Loss: 0.0062, Validation Accuracy: 0.7730
Epoch [11/20], Train Loss: 0.0016, Validation Accuracy: 0.7561
Epoch [12/20], Train Loss: 0.0005, Validation Accuracy: 0.7749
Epoch [13/20], Train Loss: 0.0015, Validation Accuracy: 0.7711
Epoch [14/20], Train Loss: 0.0039, Validation Accuracy: 0.7645
Epoch [15/20], Train Loss: 0.0005, Validation Accuracy: 0.7561
Epoch [16/20], Train Loss: 0.0000, Validation Accuracy: 0.7580
E

[I 2024-11-02 06:43:15,460] Trial 0 finished with value: 0.8076923076923077 and parameters: {'hidden_size': 256, 'num_gru_layers': 2, 'learning_rate': 0.001, 'batch_size': 32, 'num_epochs': 20, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.8076923076923077.


Epoch [20/20], Train Loss: 0.0000, Validation Accuracy: 0.7570
Epoch [1/20], Train Loss: 0.4879, Validation Accuracy: 0.7871
Epoch [2/20], Train Loss: 0.2637, Validation Accuracy: 0.7852
Epoch [3/20], Train Loss: 0.1076, Validation Accuracy: 0.7908
Epoch [4/20], Train Loss: 0.0439, Validation Accuracy: 0.7786
Epoch [5/20], Train Loss: 0.0117, Validation Accuracy: 0.7749
Epoch [6/20], Train Loss: 0.0046, Validation Accuracy: 0.7664
Epoch [7/20], Train Loss: 0.0040, Validation Accuracy: 0.7655
Epoch [8/20], Train Loss: 0.0015, Validation Accuracy: 0.7636
Epoch [9/20], Train Loss: 0.0009, Validation Accuracy: 0.7758
Epoch [10/20], Train Loss: 0.0003, Validation Accuracy: 0.7730
Epoch [11/20], Train Loss: 0.0001, Validation Accuracy: 0.7702
Epoch [12/20], Train Loss: 0.0000, Validation Accuracy: 0.7730
Epoch [13/20], Train Loss: 0.0000, Validation Accuracy: 0.7711
Epoch [14/20], Train Loss: 0.0000, Validation Accuracy: 0.7730
Epoch [15/20], Train Loss: 0.0000, Validation Accuracy: 0.7683
E

[I 2024-11-02 06:43:59,404] Trial 1 finished with value: 0.7908067542213884 and parameters: {'hidden_size': 256, 'num_gru_layers': 1, 'learning_rate': 0.001, 'batch_size': 32, 'num_epochs': 20, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.8076923076923077.


...

Epoch [10/10], Train Loss: 50.0182, Validation Accuracy: 0.5009
Epoch [1/20], Train Loss: 0.6655, Validation Accuracy: 0.6979
Epoch [2/20], Train Loss: 0.4873, Validation Accuracy: 0.7645
Epoch [3/20], Train Loss: 0.3910, Validation Accuracy: 0.7730
Epoch [4/20], Train Loss: 0.3342, Validation Accuracy: 0.7805
Epoch [5/20], Train Loss: 0.2831, Validation Accuracy: 0.7786
Epoch [6/20], Train Loss: 0.2310, Validation Accuracy: 0.7786
Epoch [7/20], Train Loss: 0.1860, Validation Accuracy: 0.7795
Epoch [8/20], Train Loss: 0.1439, Validation Accuracy: 0.7833
Epoch [9/20], Train Loss: 0.1141, Validation Accuracy: 0.7758
Epoch [10/20], Train Loss: 0.0867, Validation Accuracy: 0.7758
Epoch [11/20], Train Loss: 0.0692, Validation Accuracy: 0.7720
Epoch [12/20], Train Loss: 0.0547, Validation Accuracy: 0.7720
Epoch [13/20], Train Loss: 0.0564, Validation Accuracy: 0.7730
Epoch [14/20], Train Loss: 0.0387, Validation Accuracy: 0.7730
Epoch [15/20], Train Loss: 0.0291, Validation Accuracy: 0.7692


[I 2024-11-02 07:26:48,161] Trial 48 finished with value: 0.7833020637898687 and parameters: {'hidden_size': 128, 'num_gru_layers': 2, 'learning_rate': 0.0001, 'batch_size': 64, 'num_epochs': 20, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.8076923076923077.


Epoch [20/20], Train Loss: 0.0151, Validation Accuracy: 0.7636
Epoch [1/50], Train Loss: 0.4905, Validation Accuracy: 0.7824
Epoch [2/50], Train Loss: 0.2625, Validation Accuracy: 0.8011
Epoch [3/50], Train Loss: 0.1209, Validation Accuracy: 0.7730
Epoch [4/50], Train Loss: 0.0513, Validation Accuracy: 0.7749
Epoch [5/50], Train Loss: 0.0207, Validation Accuracy: 0.7702
Epoch [6/50], Train Loss: 0.0120, Validation Accuracy: 0.7645
Epoch [7/50], Train Loss: 0.0092, Validation Accuracy: 0.7720
Epoch [8/50], Train Loss: 0.0040, Validation Accuracy: 0.7767
Epoch [9/50], Train Loss: 0.0023, Validation Accuracy: 0.7645
Epoch [10/50], Train Loss: 0.0022, Validation Accuracy: 0.7608
Epoch [11/50], Train Loss: 0.0029, Validation Accuracy: 0.7617
Epoch [12/50], Train Loss: 0.0052, Validation Accuracy: 0.7636
Epoch [13/50], Train Loss: 0.0002, Validation Accuracy: 0.7795
Epoch [14/50], Train Loss: 0.0000, Validation Accuracy: 0.7749
Epoch [15/50], Train Loss: 0.0000, Validation Accuracy: 0.7730
E

[I 2024-11-02 07:28:08,070] Trial 49 finished with value: 0.801125703564728 and parameters: {'hidden_size': 64, 'num_gru_layers': 3, 'learning_rate': 0.001, 'batch_size': 32, 'num_epochs': 50, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.8076923076923077.


Epoch [50/50], Train Loss: 0.0000, Validation Accuracy: 0.7664


# Best Hyperparameter

In [ ]:
# The Best Hyperparameters
hidden_size = 256  # Hidden size choices
num_gru_layers = 1  # Number of GRU layers choices
learning_rate = 0.00001  # Learning rate choices
batch_size = 32  # Batch size choices
num_epochs = 40  # num epoch choices
optimizer_name = 'AdamW'  # Optimizer choices

In [ ]:
# Reset the seed to ensure fairness when comparing performances
set_seed()

# Load Data
train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

# Initialize the model
model = BiGRU(embedding_matrix, hidden_size, num_gru_layers, True, 'mhsa')
model.to(device)
# Use BCELoss for binary classification
criterion = nn.BCELoss()
# Choose optimizer based on trial suggestion
optimizer = optimizer_factory(optimizer_name, model, learning_rate)
# Initialize the early stopper
early_stopper = EarlyStopper()

# Lists to store loss and accuracy for visualization
train_losses = []
val_accuracies = []

# Training Loop
best_val_accuracy = 0
best_model = None
for epoch in range(num_epochs):
    # Train on train set
    train_loss = train_loop(train_loader, model, criterion, optimizer)
    train_losses.append(train_loss)

    # Evaluate on validation set
    val_accuracy = test_loop(val_loader, model)
    val_accuracies.append(val_accuracy)

    # Get the best model
    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_model = copy.deepcopy(model)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping
    if early_stopper.early_stop(val_accuracy):
        print("Early stopping triggered!")
        break

# Evaluate on test data
test_accuracy = test_loop(test_loader, best_model)

# Print out the performance
print(f"Using Best Model, Test Accuracy: {test_accuracy:.4f}")

Epoch [1/40], Train Loss: 0.6891, Validation Accuracy: 0.6370
Epoch [2/40], Train Loss: 0.6603, Validation Accuracy: 0.6820
Epoch [3/40], Train Loss: 0.5648, Validation Accuracy: 0.7270
Epoch [4/40], Train Loss: 0.4897, Validation Accuracy: 0.7486
Epoch [5/40], Train Loss: 0.4582, Validation Accuracy: 0.7655
Epoch [6/40], Train Loss: 0.4399, Validation Accuracy: 0.7692
Epoch [7/40], Train Loss: 0.4269, Validation Accuracy: 0.7645
Epoch [8/40], Train Loss: 0.4160, Validation Accuracy: 0.7692
Epoch [9/40], Train Loss: 0.4057, Validation Accuracy: 0.7674
Epoch [10/40], Train Loss: 0.3953, Validation Accuracy: 0.7692
Epoch [11/40], Train Loss: 0.3853, Validation Accuracy: 0.7711
Epoch [12/40], Train Loss: 0.3770, Validation Accuracy: 0.7758
Epoch [13/40], Train Loss: 0.3676, Validation Accuracy: 0.7749
Epoch [14/40], Train Loss: 0.3590, Validation Accuracy: 0.7795
Epoch [15/40], Train Loss: 0.3493, Validation Accuracy: 0.7720
Epoch [16/40], Train Loss: 0.3407, Validation Accuracy: 0.7692
E

# Archive

The sections below are for reference only.

# biGRU Hyperparameter Tuning (FastText)

In [ ]:
def bigru_objective(trial):

    max_accuracy = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Hyperparameters
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])  # Hidden size choices
    num_gru_layers = trial.suggest_categorical('num_gru_layers', [1, 2, 3])  # Number of GRU layers
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])  # Learning rate choices
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size choices
    num_epochs = trial.suggest_categorical('num_epochs', [10, 20, 30, 40, 50])  # Number of epochs
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])  # Optimizer choices

    # Load data using the dataloader_factory
    train_loader, val_loader, test_loader = dataloader_factory(X_train_OOV_sequence, X_val_OOV_sequence, X_test_OOV_sequence, Y_train_OOV, Y_val_OOV, Y_test_OOV, batch_size, device)

    # Define the BiGRU model
    model = BiGRU(fasttext_embedding_matrix, hidden_size, num_gru_layers, True, 'max_pool')
    model.to(device)
    criterion = nn.BCELoss()  # Use BCELoss for binary classification

    # Choose optimizer based on trial suggestion
    optimizer = optimizer_factory(optimizer_name, model, learning_rate)

    # Lists to store loss and accuracy for visualization
    train_losses = []
    val_accuracies = []

    # Training Loop
    for epoch in range(num_epochs):
        loss = train_loop(train_loader, model, criterion, optimizer)
        train_losses.append(loss)

        # Evaluate on validation set
        val_accuracy = test_loop(val_loader, model)
        val_accuracies.append(val_accuracy)
        if val_accuracy > max_accuracy:
            max_accuracy = val_accuracy

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    return max_accuracy  # Return validation accuracy for optimization

In [ ]:
set_seed()
study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(bigru_objective, n_trials=50)  # Invoke optimization of the objective function.

[I 2024-10-25 16:55:21,949] A new study created in memory with name: no-name-b9b681a9-d0b9-43b8-b50d-238289aecdd2


Epoch [1/20], Train Loss: 0.6810, Validation Accuracy: 0.6454
Epoch [2/20], Train Loss: 0.5771, Validation Accuracy: 0.7420
Epoch [3/20], Train Loss: 0.4389, Validation Accuracy: 0.7683
Epoch [4/20], Train Loss: 0.3658, Validation Accuracy: 0.7711
Epoch [5/20], Train Loss: 0.3055, Validation Accuracy: 0.7739
Epoch [6/20], Train Loss: 0.2590, Validation Accuracy: 0.7814
Epoch [7/20], Train Loss: 0.2116, Validation Accuracy: 0.7786
Epoch [8/20], Train Loss: 0.1842, Validation Accuracy: 0.7749
Epoch [9/20], Train Loss: 0.1612, Validation Accuracy: 0.7814
Epoch [10/20], Train Loss: 0.1428, Validation Accuracy: 0.7627
Epoch [11/20], Train Loss: 0.1118, Validation Accuracy: 0.7683
Epoch [12/20], Train Loss: 0.1012, Validation Accuracy: 0.7777
Epoch [13/20], Train Loss: 0.0824, Validation Accuracy: 0.7777
Epoch [14/20], Train Loss: 0.0699, Validation Accuracy: 0.7824
Epoch [15/20], Train Loss: 0.0622, Validation Accuracy: 0.7833
Epoch [16/20], Train Loss: 0.0573, Validation Accuracy: 0.7580
E

[I 2024-10-25 16:55:54,276] Trial 0 finished with value: 0.7833020637898687 and parameters: {'hidden_size': 256, 'num_gru_layers': 1, 'learning_rate': 0.0001, 'batch_size': 128, 'num_epochs': 20, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.7833020637898687.


Epoch [20/20], Train Loss: 0.0466, Validation Accuracy: 0.7636
Epoch [1/20], Train Loss: 0.5673, Validation Accuracy: 0.6839
Epoch [2/20], Train Loss: 0.2703, Validation Accuracy: 0.7655
Epoch [3/20], Train Loss: 0.1208, Validation Accuracy: 0.7749
Epoch [4/20], Train Loss: 0.0564, Validation Accuracy: 0.7674
Epoch [5/20], Train Loss: 0.0296, Validation Accuracy: 0.7139
Epoch [6/20], Train Loss: 0.0164, Validation Accuracy: 0.7477
Epoch [7/20], Train Loss: 0.0133, Validation Accuracy: 0.7580
Epoch [8/20], Train Loss: 0.0104, Validation Accuracy: 0.7439
Epoch [9/20], Train Loss: 0.0031, Validation Accuracy: 0.7552
Epoch [10/20], Train Loss: 0.0135, Validation Accuracy: 0.7589
Epoch [11/20], Train Loss: 0.0015, Validation Accuracy: 0.7598
Epoch [12/20], Train Loss: 0.0001, Validation Accuracy: 0.7570
Epoch [13/20], Train Loss: 0.0000, Validation Accuracy: 0.7570
Epoch [14/20], Train Loss: 0.0000, Validation Accuracy: 0.7570
Epoch [15/20], Train Loss: 0.0000, Validation Accuracy: 0.7580
E

[I 2024-10-25 16:56:26,276] Trial 1 finished with value: 0.774859287054409 and parameters: {'hidden_size': 128, 'num_gru_layers': 3, 'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 20, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.7833020637898687.


...

Epoch [30/30], Train Loss: 0.5099, Validation Accuracy: 0.7139
Epoch [1/30], Train Loss: 0.6929, Validation Accuracy: 0.5385
Epoch [2/30], Train Loss: 0.6913, Validation Accuracy: 0.5629
Epoch [3/30], Train Loss: 0.6888, Validation Accuracy: 0.5009
Epoch [4/30], Train Loss: 0.6855, Validation Accuracy: 0.5028
Epoch [5/30], Train Loss: 0.6813, Validation Accuracy: 0.5919
Epoch [6/30], Train Loss: 0.6740, Validation Accuracy: 0.6210
Epoch [7/30], Train Loss: 0.6634, Validation Accuracy: 0.6276
Epoch [8/30], Train Loss: 0.6434, Validation Accuracy: 0.5966
Epoch [9/30], Train Loss: 0.6099, Validation Accuracy: 0.7017
Epoch [10/30], Train Loss: 0.5638, Validation Accuracy: 0.7073
Epoch [11/30], Train Loss: 0.5323, Validation Accuracy: 0.7270
Epoch [12/30], Train Loss: 0.4970, Validation Accuracy: 0.7467
Epoch [13/30], Train Loss: 0.4661, Validation Accuracy: 0.7674
Epoch [14/30], Train Loss: 0.4370, Validation Accuracy: 0.7439
Epoch [15/30], Train Loss: 0.4119, Validation Accuracy: 0.7627
E

[I 2024-10-25 17:41:35,347] Trial 48 finished with value: 0.7870544090056285 and parameters: {'hidden_size': 256, 'num_gru_layers': 2, 'learning_rate': 0.01, 'batch_size': 32, 'num_epochs': 30, 'optimizer': 'SGD'}. Best is trial 22 with value: 0.7936210131332082.


Epoch [30/30], Train Loss: 0.1172, Validation Accuracy: 0.7495
Epoch [1/10], Train Loss: 0.6937, Validation Accuracy: 0.5009
Epoch [2/10], Train Loss: 0.6937, Validation Accuracy: 0.5019
Epoch [3/10], Train Loss: 0.6940, Validation Accuracy: 0.5028
Epoch [4/10], Train Loss: 0.6940, Validation Accuracy: 0.5009
Epoch [5/10], Train Loss: 0.6938, Validation Accuracy: 0.4981
Epoch [6/10], Train Loss: 0.6935, Validation Accuracy: 0.4953
Epoch [7/10], Train Loss: 0.6930, Validation Accuracy: 0.4981
Epoch [8/10], Train Loss: 0.6941, Validation Accuracy: 0.5019
Epoch [9/10], Train Loss: 0.6937, Validation Accuracy: 0.5094


[I 2024-10-25 17:41:47,163] Trial 49 finished with value: 0.5121951219512195 and parameters: {'hidden_size': 64, 'num_gru_layers': 2, 'learning_rate': 0.0001, 'batch_size': 32, 'num_epochs': 10, 'optimizer': 'SGD'}. Best is trial 22 with value: 0.7936210131332082.


Epoch [10/10], Train Loss: 0.6936, Validation Accuracy: 0.5122


# Best Hyperparameter biGRU (FastText)

In [ ]:
# The Best Hyperparameters
hidden_size = 64  # Hidden size choices
num_gru_layers = 2  # Number of GRU layers choices
learning_rate = 0.01  # Learning rate choices
batch_size = 32  # Batch size choices
num_epochs = 30  # num epoch choices
optimizer_name = 'SGD'  # Optimizer choices

In [ ]:
# Reset the seed to ensure fairness when comparing performances
set_seed()

# Load Data
train_loader, val_loader, test_loader = dataloader_factory(X_train_OOV_sequence, X_val_OOV_sequence, X_test_OOV_sequence, Y_train_OOV, Y_val_OOV, Y_test_OOV, batch_size, device)

# Initialize the model
model = BiGRU(fasttext_embedding_matrix, hidden_size, num_gru_layers, True, 'max_pool')
model.to(device)
# Use BCELoss for binary classification
criterion = nn.BCELoss()
# Choose optimizer based on trial suggestion
optimizer = optimizer_factory(optimizer_name, model, learning_rate)
# Initialize the early stopper
early_stopper = EarlyStopper()

# Lists to store loss and accuracy for visualization
train_losses = []
val_accuracies = []

# Training Loop
best_val_accuracy = 0
best_model = None
for epoch in range(num_epochs):
    # Train on train set
    train_loss = train_loop(train_loader, model, criterion, optimizer)
    train_losses.append(train_loss)

    # Evaluate on validation set
    val_accuracy = test_loop(val_loader, model)
    val_accuracies.append(val_accuracy)

    # Get the best model
    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_model = copy.deepcopy(model)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping
    if early_stopper.early_stop(val_accuracy):
        print("Early stopping triggered!")
        break

# Evaluate on test data
test_accuracy = test_loop(test_loader, best_model)

# Print out the performance
print(f"Using Best Model, Test Accuracy: {test_accuracy:.4f}")

Epoch [1/30], Train Loss: 0.6935, Validation Accuracy: 0.5441
Epoch [2/30], Train Loss: 0.6920, Validation Accuracy: 0.5019
Epoch [3/30], Train Loss: 0.6908, Validation Accuracy: 0.5657
Epoch [4/30], Train Loss: 0.6894, Validation Accuracy: 0.5722
Epoch [5/30], Train Loss: 0.6858, Validation Accuracy: 0.5629
Epoch [6/30], Train Loss: 0.6806, Validation Accuracy: 0.5385
Epoch [7/30], Train Loss: 0.6750, Validation Accuracy: 0.6069
Epoch [8/30], Train Loss: 0.6649, Validation Accuracy: 0.6266
Epoch [9/30], Train Loss: 0.6479, Validation Accuracy: 0.5985
Epoch [10/30], Train Loss: 0.6183, Validation Accuracy: 0.6220
Epoch [11/30], Train Loss: 0.5818, Validation Accuracy: 0.6820
Epoch [12/30], Train Loss: 0.5464, Validation Accuracy: 0.7073
Epoch [13/30], Train Loss: 0.5106, Validation Accuracy: 0.7467
Epoch [14/30], Train Loss: 0.4823, Validation Accuracy: 0.7345
Epoch [15/30], Train Loss: 0.4509, Validation Accuracy: 0.7598
Epoch [16/30], Train Loss: 0.4217, Validation Accuracy: 0.7430
E

# biGRU Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiGRU(nn.Module):
    def __init__(self, embedding_matrix, hidden_size=64, num_gru_layers=1, bidirectional=True, aggregation='max_pool', use_attention=True):
        super(BiGRU, self).__init__()
        embedding_dim = embedding_matrix.shape[1]
        self.aggregation = aggregation
        self.use_attention = use_attention

        # Embedding layer using the provided embedding matrix
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float32),
            padding_idx=0,  # Assuming 0 is the padding index
            freeze=False  # Freezing the embeddings
        )

        # Dropout layer
        self.dropout = nn.Dropout(0.2)

        # GRU layer(s)
        self.gru = nn.GRU(input_size=embedding_dim,
                          hidden_size=hidden_size,
                          num_layers=num_gru_layers,
                          dropout=(0 if num_gru_layers == 1 else 0.2),
                          bidirectional=bidirectional,
                          batch_first=True)

        # Attention layer
        num_directions = 2 if bidirectional else 1
        if self.use_attention:
            self.attention_weights = nn.Linear(hidden_size * num_directions, 1)

        # Linear layer for binary classification
        self.fc = nn.Linear(hidden_size * num_directions, 1)

    def forward(self, x, hidden=None):
        # Pass through the embedding layer
        embedded = self.embedding(x)

        # Pass through the GRU layer(s)
        gru_out, _ = self.gru(embedded, hidden)

        # Attention mechanism
        if self.use_attention:
            # Compute attention scores and apply softmax
            attn_scores = self.attention_weights(gru_out).squeeze(-1)  # Shape: (batch_size, seq_len)
            attn_weights = F.softmax(attn_scores, dim=1).unsqueeze(-1)  # Shape: (batch_size, seq_len, 1)

            # Weighted sum of hidden states (context vector)
            gru_out = torch.sum(gru_out * attn_weights, dim=1)  # Shape: (batch_size, hidden_size * num_directions)
        else:
            # Pooling over the time dimension (sequence length) if no attention
            if self.aggregation == 'max_pool':
                gru_out, _ = torch.max(gru_out, dim=1)  # Max-pooling over the sequence length
            elif self.aggregation == 'mean_pool':
                gru_out = torch.mean(gru_out, dim=1)  # Mean-pooling over the sequence length

        # Apply dropout
        gru_out = self.dropout(gru_out)

        # Pass through the fully connected layer and apply sigmoid activation
        out = self.fc(gru_out)
        out = torch.sigmoid(out)

        return out


# Hyperparameter Tuning

In [ ]:
def bigru_w2v_attention_objective(trial):

    max_accuracy = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Hyperparameters
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])  # Hidden size choices
    num_gru_layers = trial.suggest_categorical('num_gru_layers', [1, 2, 3])  # Number of GRU layers
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])  # Learning rate choices
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])  # Batch size choices
    num_epochs = trial.suggest_categorical('num_epochs', [10, 20, 30, 40, 50])  # Number of epochs
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop', 'AdamW'])  # Optimizer choices

    # Load data using the dataloader_factory
    train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

    # Define the BiGRU model
    model = BiGRU(embedding_matrix, hidden_size, num_gru_layers, True, 'max_pool', True)
    model.to(device)
    criterion = nn.BCELoss()  # Use BCELoss for binary classification

    # Choose optimizer based on trial suggestion
    optimizer = optimizer_factory(optimizer_name, model, learning_rate)

    # Lists to store loss and accuracy for visualization
    train_losses = []
    val_accuracies = []

    # Training Loop
    for epoch in range(num_epochs):
        loss = train_loop(train_loader, model, criterion, optimizer)
        train_losses.append(loss)

        # Evaluate on validation set
        val_accuracy = test_loop(val_loader, model)
        val_accuracies.append(val_accuracy)
        if val_accuracy > max_accuracy:
            max_accuracy = val_accuracy

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    return max_accuracy  # Return validation accuracy for optimization

In [ ]:
set_seed()
study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(bigru_w2v_attention_objective, n_trials=50)  # Invoke optimization of the objective function.

[I 2024-10-30 16:49:32,332] A new study created in memory with name: no-name-eda9e606-7706-46bc-921a-0dd41b867b55


Epoch [1/10], Train Loss: 0.6925, Validation Accuracy: 0.5047
Epoch [2/10], Train Loss: 0.6913, Validation Accuracy: 0.6735
Epoch [3/10], Train Loss: 0.6899, Validation Accuracy: 0.6717
Epoch [4/10], Train Loss: 0.6884, Validation Accuracy: 0.6285
Epoch [5/10], Train Loss: 0.6864, Validation Accuracy: 0.6557
Epoch [6/10], Train Loss: 0.6835, Validation Accuracy: 0.6585
Epoch [7/10], Train Loss: 0.6791, Validation Accuracy: 0.6773
Epoch [8/10], Train Loss: 0.6725, Validation Accuracy: 0.6745
Epoch [9/10], Train Loss: 0.6623, Validation Accuracy: 0.6914


[I 2024-10-30 16:49:50,628] Trial 0 finished with value: 0.6913696060037523 and parameters: {'hidden_size': 128, 'num_gru_layers': 2, 'learning_rate': 1e-05, 'batch_size': 64, 'num_epochs': 10, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.6913696060037523.


Epoch [10/10], Train Loss: 0.6451, Validation Accuracy: 0.6857
Epoch [1/40], Train Loss: 0.6925, Validation Accuracy: 0.5103
Epoch [2/40], Train Loss: 0.6913, Validation Accuracy: 0.6351
Epoch [3/40], Train Loss: 0.6894, Validation Accuracy: 0.6445
Epoch [4/40], Train Loss: 0.6869, Validation Accuracy: 0.6295
Epoch [5/40], Train Loss: 0.6816, Validation Accuracy: 0.6839
Epoch [6/40], Train Loss: 0.6671, Validation Accuracy: 0.6895
Epoch [7/40], Train Loss: 0.6133, Validation Accuracy: 0.7148
Epoch [8/40], Train Loss: 0.5264, Validation Accuracy: 0.7336
Epoch [9/40], Train Loss: 0.4848, Validation Accuracy: 0.7495
Epoch [10/40], Train Loss: 0.4562, Validation Accuracy: 0.7458
Epoch [11/40], Train Loss: 0.4356, Validation Accuracy: 0.7514
Epoch [12/40], Train Loss: 0.4185, Validation Accuracy: 0.7627
Epoch [13/40], Train Loss: 0.4050, Validation Accuracy: 0.7608
Epoch [14/40], Train Loss: 0.3928, Validation Accuracy: 0.7617
Epoch [15/40], Train Loss: 0.3836, Validation Accuracy: 0.7636
E

[I 2024-10-30 16:50:58,903] Trial 1 finished with value: 0.7804878048780488 and parameters: {'hidden_size': 128, 'num_gru_layers': 2, 'learning_rate': 1e-05, 'batch_size': 32, 'num_epochs': 40, 'optimizer': 'AdamW'}. Best is trial 1 with value: 0.7804878048780488.


...

Epoch [30/30], Train Loss: 0.0013, Validation Accuracy: 0.7758
Epoch [1/30], Train Loss: 0.4847, Validation Accuracy: 0.7664
Epoch [2/30], Train Loss: 0.2565, Validation Accuracy: 0.8030
Epoch [3/30], Train Loss: 0.1235, Validation Accuracy: 0.7992
Epoch [4/30], Train Loss: 0.0524, Validation Accuracy: 0.7974
Epoch [5/30], Train Loss: 0.0214, Validation Accuracy: 0.7852
Epoch [6/30], Train Loss: 0.0093, Validation Accuracy: 0.7777
Epoch [7/30], Train Loss: 0.0028, Validation Accuracy: 0.7720
Epoch [8/30], Train Loss: 0.0037, Validation Accuracy: 0.7720
Epoch [9/30], Train Loss: 0.0014, Validation Accuracy: 0.7674
Epoch [10/30], Train Loss: 0.0034, Validation Accuracy: 0.7702
Epoch [11/30], Train Loss: 0.0027, Validation Accuracy: 0.7589
Epoch [12/30], Train Loss: 0.0002, Validation Accuracy: 0.7636
Epoch [13/30], Train Loss: 0.0000, Validation Accuracy: 0.7683
Epoch [14/30], Train Loss: 0.0000, Validation Accuracy: 0.7711
Epoch [15/30], Train Loss: 0.0000, Validation Accuracy: 0.7711
E

[I 2024-10-30 17:24:57,740] Trial 48 finished with value: 0.8030018761726079 and parameters: {'hidden_size': 128, 'num_gru_layers': 1, 'learning_rate': 0.001, 'batch_size': 32, 'num_epochs': 30, 'optimizer': 'RMSprop'}. Best is trial 45 with value: 0.8058161350844277.


Epoch [30/30], Train Loss: 0.0000, Validation Accuracy: 0.7702
Epoch [1/30], Train Loss: 0.6459, Validation Accuracy: 0.7176
Epoch [2/30], Train Loss: 0.4881, Validation Accuracy: 0.7589
Epoch [3/30], Train Loss: 0.4167, Validation Accuracy: 0.7711
Epoch [4/30], Train Loss: 0.3676, Validation Accuracy: 0.7777
Epoch [5/30], Train Loss: 0.3268, Validation Accuracy: 0.7749
Epoch [6/30], Train Loss: 0.2936, Validation Accuracy: 0.7749
Epoch [7/30], Train Loss: 0.2620, Validation Accuracy: 0.7777
Epoch [8/30], Train Loss: 0.2343, Validation Accuracy: 0.7627
Epoch [9/30], Train Loss: 0.2073, Validation Accuracy: 0.7824
Epoch [10/30], Train Loss: 0.1831, Validation Accuracy: 0.7795
Epoch [11/30], Train Loss: 0.1643, Validation Accuracy: 0.7795
Epoch [12/30], Train Loss: 0.1431, Validation Accuracy: 0.7702
Epoch [13/30], Train Loss: 0.1286, Validation Accuracy: 0.7730
Epoch [14/30], Train Loss: 0.1153, Validation Accuracy: 0.7608
Epoch [15/30], Train Loss: 0.0982, Validation Accuracy: 0.7608
E

[I 2024-10-30 17:25:21,566] Trial 49 finished with value: 0.7823639774859287 and parameters: {'hidden_size': 128, 'num_gru_layers': 1, 'learning_rate': 0.0001, 'batch_size': 64, 'num_epochs': 30, 'optimizer': 'RMSprop'}. Best is trial 45 with value: 0.8058161350844277.


Epoch [30/30], Train Loss: 0.0175, Validation Accuracy: 0.7552


# Best Hyperparameter

In [ ]:
# The Best Hyperparameters
hidden_size = 128  # Hidden size choices
num_gru_layers = 1  # Number of GRU layers choices
learning_rate = 0.001  # Learning rate choices
batch_size = 64  # Batch size choices
num_epochs = 30  # num epoch choices
optimizer_name = 'RMSprop'  # Optimizer choices

In [ ]:
# Reset the seed to ensure fairness when comparing performances
set_seed()

# Load Data
train_loader, val_loader, test_loader = dataloader_factory(X_train_sequence, X_val_sequence, X_test_sequence, Y_train, Y_val, Y_test, batch_size, device)

# Initialize the model
model = BiGRU(embedding_matrix, hidden_size, num_gru_layers, True, 'max_pool', True)
model.to(device)
# Use BCELoss for binary classification
criterion = nn.BCELoss()
# Choose optimizer based on trial suggestion
optimizer = optimizer_factory(optimizer_name, model, learning_rate)
# Initialize the early stopper
early_stopper = EarlyStopper()

# Lists to store loss and accuracy for visualization
train_losses = []
val_accuracies = []

# Training Loop
best_val_accuracy = 0
best_model = None
for epoch in range(num_epochs):
    # Train on train set
    train_loss = train_loop(train_loader, model, criterion, optimizer)
    train_losses.append(train_loss)

    # Evaluate on validation set
    val_accuracy = test_loop(val_loader, model)
    val_accuracies.append(val_accuracy)

    # Get the best model
    if val_accuracy > best_val_accuracy:
      best_val_accuracy = val_accuracy
      best_model = copy.deepcopy(model)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping
    if early_stopper.early_stop(val_accuracy):
        print("Early stopping triggered!")
        break

# Evaluate on test data
test_accuracy = test_loop(test_loader, best_model)

# Print out the performance
print(f"Using Best Model, Test Accuracy: {test_accuracy:.4f}")

Epoch [1/30], Train Loss: 0.4877, Validation Accuracy: 0.7927
Epoch [2/30], Train Loss: 0.2621, Validation Accuracy: 0.7936
Epoch [3/30], Train Loss: 0.1275, Validation Accuracy: 0.7824
Epoch [4/30], Train Loss: 0.0552, Validation Accuracy: 0.7767
Epoch [5/30], Train Loss: 0.0228, Validation Accuracy: 0.7786
Epoch [6/30], Train Loss: 0.0090, Validation Accuracy: 0.7739
Epoch [7/30], Train Loss: 0.0057, Validation Accuracy: 0.7608
Epoch [8/30], Train Loss: 0.0086, Validation Accuracy: 0.7767
Epoch [9/30], Train Loss: 0.0022, Validation Accuracy: 0.7683
Epoch [10/30], Train Loss: 0.0004, Validation Accuracy: 0.7692
Epoch [11/30], Train Loss: 0.0001, Validation Accuracy: 0.7702
Epoch [12/30], Train Loss: 0.0000, Validation Accuracy: 0.7655
Early stopping triggered!
Using Best Model, Test Accuracy: 0.8114
